In [ ]:
#import matplotlib.image as mpimg
#import matplotlib.pyplot as plt
#import matplotlib.patches as patches
#from matplotlib.patches import Rectangle
#from PIL import Image
#%matplotlib inline



# Download SVHN and extract from label file

In [1]:
import imp

svhn_images = imp.load_source('svhn_images', '../modules/svhn_images.py')

In [ ]:
import os

image_path='/flat2/raw'
if not os.path.exists(image_path): 
    os.mkdir(image_path)

for data_kind in [ 'test']:
#for data_kind in ['train', 'test']:
    print "Processing %s, please give this a few minutes to complete" % data_kind

    svhn_images.download_and_extract_SVHN(data_kind, image_path, '../data/SVHN/%s' % data_kind)

    print "Finished processing %s" % data_kind

# Data Exploration

In [ ]:
image_path='/flat2/raw'

image_counts = svhn_images.count_images_by_length(image_path, data_kind='train')

print "Train data set:"
for k,v in image_counts.items():
    print "Number of %d-digit sequences=%d" %(k, v)
    
image_counts = svhn_images.count_images_by_length(image_path, data_kind='test')

print "Test data set:"
for k,v in image_counts.items():
    print "Number of %d-digit sequences=%d" %(k, v)

In [ ]:
sample_of_images = svhn_images.get_list_of_filenames(image_path, data_kind='train', batch_size=10, max_sequence_length=3)

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

for f in [ "%s/images/%s/%s" % (image_path, 'train', png) for png in sample_of_images]:
    
    plt.figure()
    img = mpimg.imread(f)
    
    print img.shape
    plt.imshow(img, cmap='Greys', interpolation='nearest')

# Explore sample after image preprocessing

In [ ]:
X, y1, y2, y3 = svhn_images.load_batch(sample_of_images, image_path, 'train', 10, 64, 3)

for i,_ in enumerate(sample_of_images):
    
    plt.figure()
    
    plt.imshow(X[i].reshape(64,64,3), cmap='Greys', interpolation='nearest')

# Load saved trained model

In [2]:
model = imp.load_source('model', '../modules/model.py')

m = model.Model()
tf_graph = m.getGraph()
session = m.init_interactive_session(tf_graph)

In [3]:
m.load_saved_model('/flat2/saved_models/SVHN_3_digit_May7', session)

# Demostrate a few learning epochs

In [4]:
import random

keep_prob=0.5
image_path='/flat2/raw'

# Total test set with 3 digits or less is 31958
NUMBER_OF_TRAIN_IMAGES = 31958
train_images_filenames = svhn_images.get_list_of_filenames(image_path, data_kind='train', batch_size=NUMBER_OF_TRAIN_IMAGES, max_sequence_length=3)

batch_size = 128

#running 10 epochs for demonstration only
for i in range(10):
    offset = random.randint(0, NUMBER_OF_TRAIN_IMAGES - batch_size)
    x, y1, y2, y3 = svhn_images.load_batch(train_images_filenames[offset:(offset+batch_size)],\
                                       image_path, 'train', 10, 64, 3)           
    m.run_training_epoch(session, x, y1, y2, y3, keep_prob=keep_prob)

# Test model against the test set

In [5]:
# Total test set with 3 digits or less is 12920
image_path='/flat2/raw'

NUMBER_OF_TEST_IMAGES = 12920

test_images_filenames = svhn_images.get_list_of_filenames(image_path, data_kind='test', batch_size=NUMBER_OF_TEST_IMAGES, max_sequence_length=3)

# Test the images against the model in a small batch at a time, to avoid memory exhaustion errors

batch_size = NUMBER_OF_TEST_IMAGES / 20
i=0
digit1 = []
digit2 = []
digit3 = []

while i < (NUMBER_OF_TEST_IMAGES - batch_size):
    batch_images = test_images_filenames[i:i+batch_size]
    X, y1, y2, y3 = svhn_images.load_batch(batch_images, image_path, 'test', 10, 64, 3)
    
    d1_acc, d2_acc, d3_acc = m.test_model(session, X, y1, y2, y3)
    digit1.append(d1_acc)
    digit2.append(d2_acc)
    digit3.append(d3_acc)
    i=i+batch_size

In [6]:
import numpy as np
print "Mean accuracy: digit_1 = %0.3f digit_2 = %0.3f digit_3 = %0.3f " % (np.array(digit1).mean(), np.array(digit2).mean(), np.array(digit3).mean())

Mean accuracy: digit_1 = 0.752 digit_2 = 0.659 digit_3 = 0.895 
